In [ ]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 30.5MB/s]


In [42]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of testing examples: 25000


In [43]:
print(vars(train_data.examples[0]))

{'text': ['I', 'did', "n't", 'even', 'know', 'this', 'was', 'originally', 'a', 'made', '-', 'for', '-', 'tv', 'movie', 'when', 'I', 'saw', 'it', ',', 'but', 'I', 'guessed', 'it', 'through', 'the', 'running', 'time', '.', 'It', 'has', 'the', 'same', 'washed', '-', 'out', 'colors', ',', 'bland', 'characters', ',', 'and', 'horrible', 'synthesized', 'music', 'that', 'I', 'remember', 'from', 'the', '80', "'s", ',', 'plus', 'a', "'", 'social', 'platform', "'", 'that', 'practically', 'screams', '"', 'Afterschool', 'special', '"', '.', 'Anyhoo.<br', '/><br', '/>Rona', 'Jaffe', "'s", '(', 'thank', 'you', ')', 'Mazes', 'and', 'Monsters', 'was', 'made', 'in', 'the', 'heyday', 'of', 'Dungeons', '&', 'Dragons', ',', 'a', 'pen', '-', 'and', '-', 'paper', 'RPG', 'that', 'took', 'the', 'hearts', 'of', 'millions', 'of', 'geeks', 'around', 'America', '.', 'I', 'count', 'myself', 'one', 'of', 'said', 'geeks', ',', 'tho', 'I', 'have', 'never', 'played', 'D&D', 'specifically', 'I', 'have', 'dabbled', 'in',

In [44]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [45]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 12250
Number of validation examples: 5250
Number of testing examples: 25000


In [46]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [47]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [48]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 141856), (',', 134002), ('.', 115388), ('and', 76495), ('a', 76058), ('of', 69877), ('to', 65801), ('is', 53244), ('in', 42890), ('I', 37791), ('it', 37066), ('that', 34456), ('"', 30772), ("'s", 29755), ('this', 29660), ('-', 25565), ('was', 24569), ('/><br', 24551), ('as', 21015), ('with', 20837)]


In [49]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [50]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [51]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [52]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [53]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [54]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [55]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [56]:
criterion = nn.BCEWithLogitsLoss()

In [57]:
model = model.to(device)
criterion = criterion.to(device)

In [92]:
def prec_rec_f1_cal(rounded_preds, true_vals):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    tp = sum((true_vals == 1) & (rounded_preds == 1))
    tn = sum((true_vals == 0) & (rounded_preds == 0))
    fn = sum((true_vals == 1) & (rounded_preds == 0))
    fp = sum((true_vals == 0) & (rounded_preds == 1))
	# return tp, tn, fp, fn
    if (tp + fp) != 0:
        precision = tp / (tp + fp)
    else:
        precision = 0
    if (tp + fn) != 0:
        recall = tp / (tp + fn)
    else:
        recall = 0
    if (precision + recall) != 0:
        f1_score = precision * recall / (precision + recall)
    else: 
        f1_score = 0
    return precision, recall, f1_score


In [112]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    precision, recall, f1_score = prec_rec_f1_cal(rounded_preds, y)

    acc = correct.sum() / len(correct)
    return acc, precision, recall, f1_score

In [113]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1_score = 0

    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc, precision, recall, f1_score = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1_score += f1_score
    return epoch_loss / len(iterator), epoch_acc / len(iterator),  epoch_precision / len(iterator),  epoch_recall / len(iterator), epoch_f1_score / len(iterator)

In [114]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1_score = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc, precision, recall, f1_score = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1_score += f1_score        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),  epoch_precision / len(iterator),  epoch_recall / len(iterator), epoch_f1_score / len(iterator)

In [115]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [122]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_precision, train_recall, train_f1_score = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, val_precision, val_recall, val_f1_score = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%  | Train Precision: {train_precision*100:.2f}%  | Train Recall: {train_recall*100:.2f}%  | Train F1 Score: {train_f1_score*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%  | Val Precision: {val_precision*100:.2f}%    | val Recall: {val_recall*100:.2f}%   | Val F1 Score: {val_f1_score*100:.2f}%')
    

Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.693 | Train Acc: 50.33%  | Train Precision: 38.14%  | Train Recall: 9.80%  | Train F1 Score: 3.89%
	 Val. Loss: 0.695 |  Val. Acc: 48.31%  | Val Precision: 41.98%    | val Recall: 14.65%   | Val F1 Score: 10.39%
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.693 | Train Acc: 50.39%  | Train Precision: 35.07%  | Train Recall: 3.68%  | Train F1 Score: 1.88%
	 Val. Loss: 0.695 |  Val. Acc: 48.46%  | Val Precision: 40.46%    | val Recall: 11.19%   | Val F1 Score: 8.26%
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 0.693 | Train Acc: 50.63%  | Train Precision: 37.50%  | Train Recall: 1.19%  | Train F1 Score: 1.16%
	 Val. Loss: 0.695 |  Val. Acc: 48.42%  | Val Precision: 41.16%    | val Recall: 11.55%   | Val F1 Score: 8.50%
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.693 | Train Acc: 50.12%  | Train Precision: 35.48%  | Train Recall: 13.92%  | Train F1 Score: 5.14%
	 Val. Loss: 0.695 |  Val. Acc: 48.44%  | Val Precision: 40.26%    | val Recall: 1

In [123]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc, test_precision, test_recall, test_f1_score = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | test Precision: {test_precision*100:.2f}%  | test Recall: {test_recall*100:.2f}%  | Test F1 Score: {test_f1_score*100:.2f}%')
    

Test Loss: 0.685 | Test Acc: 59.81% | test Precision: 63.37%  | test Recall: 54.19%  | Test F1 Score: 27.34%
